# Russian River Step 3 -- 2D mesh

Form the 2D mesh, elevate via a DEM, condition.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# setting up logging first or else it gets preempted by another package
import watershed_workflow.ui
watershed_workflow.ui.setup_logging(1)

In [ ]:
import os,sys
import logging
import numpy as np
from matplotlib import pyplot as plt
import pickle
import shapely
import pandas as pd
import geopandas as gpd
pd.options.display.max_columns = None

import watershed_workflow 
import watershed_workflow.config
import watershed_workflow.sources
import watershed_workflow.mesh
import watershed_workflow.regions
import watershed_workflow.condition

# set the default figure size for notebooks
plt.rcParams["figure.figsize"] = (8, 6)

## Input: Parameters and other source data

In [ ]:
# Force Watershed Workflow to pull data from this directory rather than a shared data directory.
# This picks up the Coweeta-specific datasets set up here to avoid large file downloads for 
# demonstration purposes.
#
def splitPathFull(path):
    """
    Splits an absolute path into a list of components such that
    os.path.join(*splitPathFull(path)) == path
    """
    parts = []
    while True:
        head, tail = os.path.split(path)
        if head == path:  # root on Unix or drive letter with backslash on Windows (e.g., C:\)
            parts.insert(0, head)
            break
        elif tail == path:  # just a single file or directory
            parts.insert(0, tail)
            break
        else:
            parts.insert(0, tail)
            path = head
    return parts

cwd = splitPathFull(os.getcwd())
assert cwd[-1] == 'workflow'
cwd = cwd[:-1]

# Note, this directory is where downloaded data will be put as well
data_dir = os.path.join(*(cwd + ['input_data',]))
def toInput(filename):
    return os.path.join(data_dir, filename)

output_dir = os.path.join(*(cwd + ['output_data',]))
output_filenames = dict()
def fromOutput(filename):
    return os.path.join(output_dir, filename)    

def toOutput(role, filename):
    output_filenames[role] = filename
    return fromOutput(filename)

# check output and input dirs exist
if not os.path.isdir(data_dir):
    os.makedirs(data_dir, exist_ok=True)
if not os.path.isdir(output_dir):
    os.makedirs(output_dir, exist_ok=True)
       

In [ ]:
# Set the data directory to the local space to get the locally downloaded files
# REMOVE THIS CELL for general use outside fo Coweeta
watershed_workflow.config.setDataDirectory(data_dir)


In [ ]:
## Parameters cell -- this provides all parameters that can be changed via pipelining to generate a new watershed. 
name = 'RussianRiver'
hucs = ['18010110'] # a list of HUCs to run


# -- parameters to clean and reduce the river network prior to meshing
prune_by_area = 20               # km^2
simplify = 200                   # length scale to target average edge 

# -- mesh triangle refinement control
refine_d0 = 200
refine_d1 = 600

refine_L0 = 200
refine_L1 = 500

refine_A0 = refine_L0**2 / 2
refine_A1 = refine_L1**2 / 2


# Refine triangles if they get too acute
min_angle = 20 # degrees

# width of reach by stream order (order:width)
river_widths = dict({1:10, 2:10, 3:20, 4:30, 5:30}) 


# Note that, by default, we tend to work in the DayMet CRS because this allows us to avoid
# reprojecting meteorological forcing datasets.
crs = watershed_workflow.crs.default_crs

## Reload data

In [ ]:
with open(fromOutput('02_watersheds.pickle'), 'rb') as fid:
    watersheds = pickle.load(fid)

reaches = gpd.read_parquet(fromOutput('02_rivers.parquet'))
rivers = watershed_workflow.river_tree.createRivers(reaches, method='native')


In [ ]:
# this generates a zoomable map, showing different reaches and watersheds, 
# with discrete points.  Problem areas are clickable to get IDs for manual
# modifications.
m = watersheds.explore(marker=True, marker_size=10)

for river in rivers:
    m = river.explore(m=m, color='black', name=river['name'], marker=True, marker_size=10)

m = watershed_workflow.makeMap(m)
m

## Generate the mesh


In [ ]:
m2, areas, dists = watershed_workflow.tessalateRiverAligned(watersheds, rivers, river_width = river_widths,
                                             refine_min_angle = min_angle, refine_distance = [refine_d0, refine_A0, refine_d1, refine_A1],
                                             diagnostics=True, debug=True)

In [ ]:
# pre-partition the mesh 
#print(m2.num_cells)
#print(m2.num_cells * 10 / 4000)
#
## let's use 192, 3 * 64
#m2 = m2.partition(192, True)

## Get a DEM


In [ ]:
dem = watershed_workflow.sources.dem_sources['3DEP 30m'].getDataset(watersheds.df)['dem']

# provide surface mesh elevations
watershed_workflow.elevate(m2, dem)

In [ ]:
# hydrologically condition the mesh, removing pits
river_mask = np.zeros((len(m2.conn)))
for i, elem in enumerate(m2.conn):
    if not len(elem) == 3:
        river_mask[i] = 1     
watershed_workflow.condition.fillPitsDual(m2, is_waterbody=river_mask)

In [ ]:
# Plot the DEM raster
fig, ax = plt.subplots(figsize=(10, 8))

# Plot the DEM data
im = dem.plot(ax=ax, cmap='terrain', add_colorbar=False)

# Add colorbar
cbar = plt.colorbar(im, ax=ax, shrink=0.8)
cbar.set_label('Elevation (m)', rotation=270, labelpad=15)

# Add title and labels
ax.set_title('Digital Elevation Model (DEM)', fontsize=14, fontweight='bold')
ax.set_xlabel('X Coordinate')
ax.set_ylabel('Y Coordinate')

# Set equal aspect ratio
ax.set_aspect('equal')

plt.tight_layout()
plt.show()

In [ ]:
m2.labeled_sets

In [ ]:
print(watersheds.df.keys())


In [ ]:
# add regions for each polygon
watershed_workflow.regions.addWatershedAndOutletRegions(m2, watersheds, 500)

In [ ]:
# add regions for each stream order
watershed_workflow.regions.addStreamOrderRegions(m2, rivers)


In [ ]:
def printRed(txt):
    print("\033[31m", txt, "\033[0m")

print('2D labeled sets')
print('---------------')
for ls in m2.labeled_sets:
    printer = print
    if len(ls.ent_ids) == 0:
        printer = printRed
    printer(f'{ls.setid} : {ls.entity} : {len(ls.ent_ids)} : "{ls.name}"')

In [ ]:
# save the mesh and regions
with open(toOutput('m2', '03_m2.pickle'), 'wb') as fid:
    pickle.dump(m2, fid)

In [ ]:
# lastly, reread, update, and output filenames
with open(toOutput('03_output_filenames', '03_output_filenames.txt'), 'wb') as fid:
    pickle.dump(output_filenames, fid)